In [62]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [63]:
!kaggle datasets download -d ananthu017/emotion-detection-fer

Dataset URL: https://www.kaggle.com/datasets/ananthu017/emotion-detection-fer
License(s): CC0-1.0
emotion-detection-fer.zip: Skipping, found more recently modified local copy (use --force to force download)


In [64]:
import zipfile
zip_ref = zipfile.ZipFile('/content/emotion-detection-fer.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [65]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing import image

In [66]:
train_dir = '/content/train'
test_dir = '/content/test'

In [67]:
#Defining Parameters
img_size = (128, 128)
batch_size = 32
epochs = 20
num_classes = 7  # angry, disgusted, fearful, happy, neutral, sad, surprised

Load Images

In [68]:
#Data Augmentation on train data to increase accuracy
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range = 10,
width_shift_range=0.1,
height_shift_range=0.1,
horizontal_flip = True
)

In [69]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [70]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [71]:
##Build the CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(img_size[0], img_size[1], 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(train_data.num_classes, activation='softmax')
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,484,807 (32.37 MB)

 Trainable params: 8,484,359 (32.37 MB)

 Non-trainable params: 448 (1.75 KB)

In [73]:
#Train the Model
model.fit(train_data, validation_data=test_data, epochs=epochs)

Epoch 1/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 149s 159ms/step - accuracy: 0.2818 - loss: 1.9475 - val_accuracy: 0.4009 - val_loss: 1.5541
Epoch 2/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 195s 154ms/step - accuracy: 0.3668 - loss: 1.6137 - val_accuracy: 0.4348 - val_loss: 1.5299
Epoch 3/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 137s 152ms/step - accuracy: 0.4013 - loss: 1.5441 - val_accuracy: 0.4387 - val_loss: 1.4497
Epoch 4/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 134s 144ms/step - accuracy: 0.4285 - loss: 1.4821 - val_accuracy: 0.4645 - val_loss: 1.3976
Epoch 5/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 132s 147ms/step - accuracy: 0.4421 - loss: 1.4478 - val_accuracy: 0.4741 - val_loss: 1.3605
Epoch 6/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 143s 148ms/step - accuracy: 0.4555 - loss: 1.4098 - val_accuracy: 0.5032 - val_loss: 1.3106
Epoch 7/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 132s 147ms/step - accuracy: 0.4748 - loss: 1.3701 - val_accuracy: 0.5132 - val_loss: 1.2798
Epoch 8/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 132s 147ms/step - accuracy: 0.4852 -

In [74]:
#Save the model
model.save('/content/emotion_cnn_model.h5')

Prediction

In [75]:
from tensorflow.keras.preprocessing import image
class_labels = list(train_data.class_indices.keys())

In [77]:
img_path = '/content/angry.jpg'
img = image.load_img(img_path, target_size=img_size)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

In [78]:
pred = model.predict(img_array)
print("Predicted emotion:", class_labels[np.argmax(pred)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 841ms/step
Predicted emotion: sad


In [ ]:
##The model somehow performs better when you don't apply batch normalization layer and don't perform data augmentation